# Happiness using TPOT
Of course, what I'm going to perform here below has no real meaning. The happinness score is totally based on the features in the dataset. But, that is not going to matter to me since I want to try TPOT for once to see how it performs on small datasets and what the cool tricks are.

Perhaps I'll do another kernel to make some visuals using plotly


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/2015.csv')

In [ ]:
df.head()

In [ ]:
df.shape

Let's produce just some graphs, perhaps I can find something interesting.

In [ ]:
plt.scatter(np.sort(df['Economy (GDP per Capita)'].values), df['Happiness Score'])

In [ ]:
plt.scatter(np.sort(df['Health (Life Expectancy)'].values), df['Happiness Score'])

In [ ]:
plt.scatter(np.sort(df['Freedom'].values), df['Happiness Score'])

In [ ]:
plt.scatter(np.sort(df['Freedom'].values), df['Happiness Score'])

In [ ]:
plt.scatter(np.sort(df['Trust (Government Corruption)'].values), df['Happiness Score'])

Hmmm, nothing really shocking here except that 'Trust' in the government does not really follow a linear path over the Happiness score range.

# Some feature preprocessing
checking if country name, so not the features of the country, helps for happiness score. Don't expect it to be but you never know. Imagine a country where its citizens are really proud but also have low health and a corrupt government... 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(df['Country'])
Country1 = pd.DataFrame({'Country1': le.transform(df['Country'])})
df = pd.concat([df, Country1], axis=1)

Checking the correlations in a square and in a rank.

In [ ]:
corrmat = df.corr()
plt.subplots(figsize=(4,4))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr()[['Happiness Score']].sort_values('Happiness Score'), annot=True)

# Above
You can see that Country which we labelled to in the new column under Country1 is not really correlated. -1 and 1 is high correlation, 0 is low correlation. 

# Below
let's make the statement above visible. Highly correlated features tend to rise as much as the target. The target is in this case the happiness score. Let's see if happiness score rises as much as the highly correlated features. 

Also, rising with 1 is as much as decending is with 1 (or just say -1).

In [ ]:
plt.scatter(df['Happiness Score'], df['Economy (GDP per Capita)'])
plt.show()

Above: it is already possible to see that the economy rises together with the happiness score. We can make that even better visible with seaborn as I'll do below:

In [ ]:
sns.regplot(x = df['Happiness Score'], y = df['Economy (GDP per Capita)'])

In [ ]:
sns.regplot(x = df['Happiness Score'], y = df['Family'])

In [ ]:
sns.regplot(x = df['Happiness Score'], y = df['Country1'])

In [ ]:
sns.regplot(x = df['Happiness Score'], y = df['Standard Error'])

In [ ]:
sns.regplot(x = df['Happiness Score'], y = df['Happiness Rank'])

Well well well, are Happiness score and rank correlated or not?

# labelling region
You never know about these regions...

In [ ]:
sns.boxplot(df['Happiness Score'], df['Region'] )

I guess this is going to be interesting. Let's label these regions as quick as possible to see what the correlation is!

In [ ]:
le.fit(df['Region'])
Region1 = pd.DataFrame({'Region1': le.transform(df['Region'])})
df = pd.concat([df, Region1], axis=1)

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr()[['Happiness Score']].sort_values('Happiness Score'), annot=True)

In [ ]:
sns.regplot(x = df['Region1'], y = df['Happiness Score'])

That was disappointing... Well, whatever, let's continue.

In [ ]:
df.head()

# Feature preparation
Let's prepare it for a TPOT where the target is going to be the Happiness Score, I mean that is important right? Eventhough it would be cooler if you could predict someones country but I guess training such a model would only work if you'd overfit it. Maybe I'll try that later.

Also, I'll exclude the following feature: Dystopia Residual. Dystopia makes be depressed.
And the feature: Standard Error

In [ ]:
X = df.as_matrix([['Region1', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom',
                  'Trust (Government Corruption)', 'Generosity']])
y = df['Happiness Score'] #regression job
#y2 = df['Country1'] #classification job

In [ ]:

print(y.shape)


# Model selection using TPOT

In [ ]:
from tpot import TPOTRegressor, TPOTClassifier

In [ ]:
TPOTReg = TPOTRegressor(generations=10, population_size=50, verbosity=2)

In [ ]:
TPOTReg.fit(X, y)

In [ ]:
TPOTReg.score(X, y)

In [ ]:
y_pred = TPOTReg.predict(X)

In [ ]:
plt.plot(y, label='True')
plt.plot(y_pred, label='pred')
plt.legend()
plt.show()

# Results
Wow, okay so that totally (I mean, not totally but, really almost) matches the true line if you are able to see through the volatile prediction line over the true line. This looks overfitted but, let's check this with the 2016 scores! That's right, you probably thought I'd miss the other csv files but, I planned to train my model using the '15 data and now let's validate the trained model on the 2016 data set

In [ ]:
dft = pd.read_csv('../input/2016.csv')

In [ ]:
le.fit(dft['Region'])
Region1 = pd.DataFrame({'Region1': le.transform(dft['Region'])})
df = pd.concat([dft, Region1], axis=1)

In [ ]:
X_val = dft.as_matrix([['Region1', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom',
                  'Trust (Government Corruption)', 'Generosity']])
y_val = dft['Happiness Score']

In [ ]:
y_val_pred = TPOTReg.predict(X_val)

In [ ]:
y_val_pred

In [ ]:
y_val.ravel(157,)

Error function mean squared error from SciKit. 
I think it is the same method used by TPOT.score.


In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val, y_val_pred)

In [ ]:
plt.plot(y_val, label='True')
plt.plot(y_val_pred, label='pred')
plt.legend()
plt.show()

# Conclusion
Eventhough the prediction for the Happiness Score has a serious mean squared error, I think I managed to perform an ok-ish ML exercise. By training a model using data from 2015 and then by validating the trained model on new data which the model has never seen before, namely the 2016 data, you see it approaches the true line.

The question is, did I just approach the function for this happiness score? I believe the answer is No, given the limited amount of entry points. Perhaps it would be much better if I combined all the .

In the end, I think TPOT makes life very easy since it matches ML algo's in a very convenient way. Using different models to see which one performs the best can be a time consuming effort. Of course, ML is way more science than exibited in this notebook but I feel happy about 'cause that means there is much more to learn!